In [9]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import csv
from pprint import pprint
from datetime import date
import json
import calendar
import scipy.stats as stats

In [10]:
def run_anova(df, indexcol, pivotcol):
    # Receives the dataframe, the index column and the pivot column
    # and create a new pivoted dataframe  to be used to create the groups
    # that will be compared in the Anova test
    
    # Group by both index and pivot columns and count the number of crimes
    df_anova = df.groupby([pivotcol,indexcol])["ObjectId"].agg([("MCI count","count")]).\
                reset_index(None)
    
    # Set the index to indexcol                                          
    df_anova = df_anova.set_index(indexcol)
    
    # Pivot the grouped by dataframe on the pivotcol
    df_pivot = df_anova.pivot(columns=pivotcol,values="MCI count")
    
    # Return both the dataframe and the pvalue for all the groups of values
    return df_pivot, \
           stats.f_oneway(*[[df_pivot[col].iloc[row] for col in df_pivot.columns] \
                            for row in range(len(df_pivot))])[1]

In [11]:
##Read the Data from already processed datafile
mci_df = pd.read_csv("Resources/MCI_2014_to_2018_full.csv")[['ObjectId','MCI','occurrencedate','premisetype','Neighbourhood','wardnumber', 'wardname','wardmedian']]
mci_df.reset_index(None)
mci_df.head()

,ObjectId,MCI,occurrencedate,premisetype,Neighbourhood,wardnumber,wardname,wardmedian
0,99462,Break and Enter,2015-04-08T17:15:00.000Z,House,Long Branch (19),Ward 6,Etobicoke-Lakeshore,66350.0
1,140047,Assault,2016-03-06T15:00:00.000Z,House,Long Branch (19),Ward 6,Etobicoke-Lakeshore,66350.0
2,2404,Assault,2014-06-14T20:00:00.000Z,Outside,Long Branch (19),Ward 6,Etobicoke-Lakeshore,66350.0
3,48420,Assault,2015-08-13T03:30:00.000Z,Outside,Long Branch (19),Ward 6,Etobicoke-Lakeshore,66350.0
4,63103,Assault,2016-08-22T23:45:00.000Z,Outside,Long Branch (19),Ward 6,Etobicoke-Lakeshore,66350.0
